In [1]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ ПРОИСХОДИТ ОБЪЕДИНЕНИЕ БЕЗ СОБЫТИЙ ПЕРВОГО ПРОТЕСТНОГО ДАТАСЕТА/ВТОРОГО ПРОТЕСТНОГО ДАТАСЕТА, ТОЛЬКО РОССТАТ


In [ ]:
dorogi = pd.read_csv('дороги_new.csv') 
zhile = pd.read_csv('жилье_new.csv') 
kan = pd.read_csv('канализация_new.csv') 
vod = pd.read_csv('водопровод_new.csv') 
etna = pd.read_csv('этно_рус_new.csv') 
rab = pd.read_csv('население_работа_new.csv') 
urban = pd.read_csv('население_город_new.csv') 
income_reg = pd.read_csv('доходы_душа_new.csv') 
income_mun = pd.read_csv('доходы_муни_new.csv') 

LEAVE_PATTERN = re.compile(
    r'^(?:Городские\s+округа|Муниципальные\s+районы|'
    r'Городские\s+поселения|Сельские\s+поселения)\b',
    re.I
)

SERVICE_PATTERN = re.compile(r'''
    # сначала самые специфичные
    \bмуниципальный\s+(?:округ|район)\b  |  # Муниципальный округ/район
    \bгородской\s+округ\b               |  # Городской округ

    # им. и сельсовет/село
    \bим\.\s*                            |  
    \bсельсовет\b                       |
    \bсело\b                            |

    # номера и спецкод
    \b\d+[-–]й\b                        |
    \bUD\b                              |

    # остальные муниципальные образования
    \bмуниципальное?\s+образование\b    |
    \bмуниципалитет\b                   |
    \bмо\b                              |
    \bр[-–]?н\.?                        |
    (?<=\s)район$                       |
    (?<=\s)округ$                       |

    # субъекты федерации
    \bобласть\b                         |
    \bкрай\b                            |
    \bреспублика\b                      |
    \bфедеральный\s+округ\b             |
    \bфедеральные\s+округа\b            |
    \bавтономный\s+округ\b(?:\s*-\s*)?  |
    \bавтономная\s+область\b            |

    # посёлки, станицы, ЗАТО
    \bпос[её]л[о]к(?:\s+городского\s+типа)? |
    \bрабочий\s+пос[её]л[о]к             |
    \bстаница\b                          |
    \bЗАТО\b                             |

    # город и аббревиатуры
    \bгород(?:-герой|-курорт)?\b        |  # только отдельное слово
    \bг\.?(?=\s|$)                      |  
    \bгор\.(?=\s|$)                     |

    # в том числе:
    \bв\s+том\s+числе:
''', re.I | re.X)

def extract_toponym(series: pd.Series) -> pd.Series:

    s = series.fillna('')

    leave = s.str.match(LEAVE_PATTERN)

    # базовая очистка
    cleaned = (
        s
        .str.replace(SERVICE_PATTERN, '', regex=True)  # убираем служ. слова
        .str.replace(r'\s*\([^)]*\)', '', regex=True)  # круглые скобки
        .str.replace(r'\s{2,}', ' ', regex=True)       # пробелы
        .str.strip()                                   # обрезаем по краям
    )

    # если начинается на 'имени ', весь остаток
    mask_imeni = cleaned.str.match(r'(?i)^имени\s+')

    # первое слово
    first = cleaned.str.split(n=1).str.get(0).fillna('')
    result = first.where(~mask_imeni, cleaned)
    result = result.where(~leave, s)

    return result

dorogi['municipality'] = extract_toponym(dorogi['municipality'])
zhile['municipality']  = extract_toponym(zhile['municipality'])
kan['municipality']    = extract_toponym(kan['municipality'])
vod['municipality']    = extract_toponym(vod['municipality'])

etna['municipality'] = extract_toponym(etna['municipality'])
rab['municipality']  = extract_toponym(rab['municipality'])
urban['municipality']    = extract_toponym(urban['municipality'])
income_mun['municipality']    = extract_toponym(income_mun['municipality'])

dorogi['region_name'] = extract_toponym(dorogi['region_name'])
zhile['region_name']  = extract_toponym(zhile['region_name'])
kan['region_name']    = extract_toponym(kan['region_name'])
vod['region_name']    = extract_toponym(vod['region_name'])

income_reg['region_name'] = extract_toponym(income_reg['region_name'])
etna['region_name'] = extract_toponym(etna['region_name'])
rab['region_name']  = extract_toponym(rab['region_name'])
urban['region_name']    = extract_toponym(urban['region_name'])
income_mun['region_name']    = extract_toponym(income_mun['region_name'])




dfs1 = [
    zhile[['year', 'region_name', 'municipality', 'oktmo', 'Жилье (1000 м2)']],
    dorogi[['year', 'region_name',  'municipality', 'oktmo', 'Дороги (%)']],
    kan[['year', 'region_name',  'municipality', 'oktmo', 'Канализация (1 м)']],
    vod[['year', 'region_name',  'municipality', 'oktmo', 'Водопровод (1 метр)']],
rab[['year', 'region_name', 'municipality','oktmo', 'active_percent']],
urban[['year', 'region_name',  'municipality', 'oktmo', 'city_percent']],
income_mun[['year', 'region_name',  'municipality', 'oktmo', 'value']]
]





comb1 = reduce(lambda left, right: pd.merge(
    left, right, on=['year', 'municipality', 'oktmo', 'region_name'], how='outer'
), dfs1)

dfs2 = [etna[['municipality', 'oktmo', 'Процент (рус)', 'region_name']], comb1]

comb2 = reduce(lambda left, right: pd.merge(
    left, right, on=['municipality', 'oktmo', 'region_name'], how='outer'
), dfs2)

comb2.fillna(0, inplace=True)
comb2 = comb2.reset_index(drop=True)
comb2.to_csv('росстат_общий.csv', index=False)
comb2

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value
0,,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0
1,,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0
2,,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0
3,,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0
4,,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
179007,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0
179008,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0
179009,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0
179010,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0


ДИЗАЙН ДАННЫХ ДЛЯ МОДЕЛИ СТАНДАРТНОГО МАЧИНГА (ПО ИНФРАСТРУКТУРЕ)

In [2]:
df = pd.read_csv("data_Y48106001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Дороги (%)'})
df.to_csv("дороги_new2.csv", index=False)
dorogi = pd.read_csv('дороги_new.csv') 

df_all = pd.concat([df, dorogi], ignore_index=True)
df_all.to_csv("дороги_pow.csv", index=False)

In [3]:
dorogi = pd.read_csv('дороги_pow.csv') 
zhile = pd.read_csv('жилье_new.csv') 
kan = pd.read_csv('канализация_new.csv') 
vod = pd.read_csv('водопровод_new.csv') 


etna = pd.read_csv('этно_рус_new.csv') 
rab = pd.read_csv('население_работа_new.csv') 
urban = pd.read_csv('население_город_new.csv') 
income_reg = pd.read_csv('доходы_душа_new.csv') 
income_mun = pd.read_csv('доходы_муни_new.csv') 


LEAVE_PATTERN = re.compile(
    r'^(?:Городские\s+округа|Муниципальные\s+районы|'
    r'Городские\s+поселения|Сельские\s+поселения)\b',
    re.I
)

SERVICE_PATTERN = re.compile(r'''
    # сначала самые специфичные
    \bмуниципальный\s+(?:округ|район)\b  |  # Муниципальный округ/район
    \bгородской\s+округ\b               |  # Городской округ

    # им. и сельсовет/село
    \bим\.\s*                            |  
    \bсельсовет\b                       |
    \bсело\b                            |

    # номера и спецкод
    \b\d+[-–]й\b                        |
    \bUD\b                              |

    # остальные муниципальные образования
    \bмуниципальное?\s+образование\b    |
    \bмуниципалитет\b                   |
    \bмо\b                              |
    \bр[-–]?н\.?                        |
    (?<=\s)район$                       |
    (?<=\s)округ$                       |

    # субъекты федерации
    \bобласть\b                         |
    \bкрай\b                            |
    \bреспублика\b                      |
    \bфедеральный\s+округ\b             |
    \bфедеральные\s+округа\b            |
    \bавтономный\s+округ\b(?:\s*-\s*)?  |
    \bавтономная\s+область\b            |

    # посёлки, станицы, ЗАТО
    \bпос[её]л[о]к(?:\s+городского\s+типа)? |
    \bрабочий\s+пос[её]л[о]к             |
    \bстаница\b                          |
    \bЗАТО\b                             |

    # город и аббревиатуры
    \bгород(?:-герой|-курорт)?\b        |  # только отдельное слово
    \bг\.?(?=\s|$)                      |  
    \bгор\.(?=\s|$)                     |

    # в том числе:
    \bв\s+том\s+числе:
''', re.I | re.X)

def extract_toponym(series: pd.Series) -> pd.Series:

    s = series.fillna('')

    leave = s.str.match(LEAVE_PATTERN)

    # базовая очистка
    cleaned = (
        s
        .str.replace(SERVICE_PATTERN, '', regex=True)  # убираем служ. слова
        .str.replace(r'\s*\([^)]*\)', '', regex=True)  # круглые скобки
        .str.replace(r'\s{2,}', ' ', regex=True)       # пробелы
        .str.strip()                                   # обрезаем по краям
    )

    # если начинается на 'имени ', весь остаток
    mask_imeni = cleaned.str.match(r'(?i)^имени\s+')

    # первое слово
    first = cleaned.str.split(n=1).str.get(0).fillna('')
    result = first.where(~mask_imeni, cleaned)
    result = result.where(~leave, s)

    return result

dorogi['municipality'] = extract_toponym(dorogi['municipality'])
zhile['municipality']  = extract_toponym(zhile['municipality'])
kan['municipality']    = extract_toponym(kan['municipality'])
vod['municipality']    = extract_toponym(vod['municipality'])


etna['municipality'] = extract_toponym(etna['municipality'])
rab['municipality']  = extract_toponym(rab['municipality'])
urban['municipality']    = extract_toponym(urban['municipality'])
income_mun['municipality']    = extract_toponym(income_mun['municipality'])

dorogi['region_name'] = extract_toponym(dorogi['region_name'])
zhile['region_name']  = extract_toponym(zhile['region_name'])
kan['region_name']    = extract_toponym(kan['region_name'])
vod['region_name']    = extract_toponym(vod['region_name'])


income_reg['region_name'] = extract_toponym(income_reg['region_name'])
etna['region_name'] = extract_toponym(etna['region_name'])
rab['region_name']  = extract_toponym(rab['region_name'])
urban['region_name']    = extract_toponym(urban['region_name'])
income_mun['region_name']    = extract_toponym(income_mun['region_name'])


dfs1 = [
    zhile[['year', 'region_name', 'municipality', 'oktmo', 'Жилье (1000 м2)']],
    dorogi[['year', 'region_name',  'municipality', 'oktmo', 'Дороги (%)']],
    kan[['year', 'region_name',  'municipality', 'oktmo', 'Канализация (1 м)']],
    vod[['year', 'region_name',  'municipality', 'oktmo', 'Водопровод (1 метр)']],

rab[['year', 'region_name', 'municipality','oktmo', 'active_percent']],
urban[['year', 'region_name',  'municipality', 'oktmo', 'city_percent']],
income_mun[['year', 'region_name',  'municipality', 'oktmo', 'value']],
]





comb1 = reduce(lambda left, right: pd.merge(
    left, right, on=['year', 'municipality', 'oktmo', 'region_name'], how='outer'
), dfs1)

dfs2 = [etna[['municipality', 'oktmo', 'Процент (рус)', 'region_name']], comb1]

comb2 = reduce(lambda left, right: pd.merge(
    left, right, on=['municipality', 'oktmo', 'region_name'], how='outer'
), dfs2)

comb2.fillna(0, inplace=True)
comb2 = comb2.reset_index(drop=True)
comb2.to_csv('росстат_общий_new.csv', index=False)
comb2

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value
0,,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0
1,,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0
2,,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0
3,,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0
4,,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
179341,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0
179342,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0
179343,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0
179344,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0


ДИЗАЙН МАЧИНГА ДЛЯ НЕСТАНДАРТНОГО МАЧИНГА (+)

In [4]:
dorogi = pd.read_csv('дороги_pow.csv') 
zhile = pd.read_csv('жилье_new.csv') 
kan = pd.read_csv('канализация_new.csv') 
vod = pd.read_csv('водопровод_new.csv') 


nas = pd.read_csv('население.csv') 
dem = pd.read_csv('убыль.csv') 
ras = pd.read_csv('расходы_муницип.csv')
vpotr = pd.read_csv('вода_потреб.csv')
tpotr = pd.read_csv('тепло_потреб.csv')
epotr = pd.read_csv('электро_потреб.csv')
inf = pd.read_csv('нет_дорог.csv')
rev = pd.read_csv('прибыль.csv')

bud = pd.read_csv('бюджет.csv') 
rod = pd.read_csv('рождаемость.csv') 
ded = pd.read_csv('смертность.csv')
deti = pd.read_csv('роды.csv')
inv = pd.read_csv('инвест.csv')
srnas = pd.read_csv('среднегоднас.csv')
kvart = pd.read_csv('квартиры.csv')
zar = pd.read_csv('зарплата.csv')
active = pd.read_csv('земля.csv')

LEAVE_PATTERN = re.compile(
    r'^(?:Городские\s+округа|Муниципальные\s+районы|'
    r'Городские\s+поселения|Сельские\s+поселения)\b',
    re.I
)

SERVICE_PATTERN = re.compile(r'''
    # сначала самые специфичные
    \bмуниципальный\s+(?:округ|район)\b  |  # Муниципальный округ/район
    \bгородской\s+округ\b               |  # Городской округ

    # им. и сельсовет/село
    \bим\.\s*                            |  
    \bсельсовет\b                       |
    \bсело\b                            |

    # номера и спецкод
    \b\d+[-–]й\b                        |
    \bUD\b                              |

    # остальные муниципальные образования
    \bмуниципальное?\s+образование\b    |
    \bмуниципалитет\b                   |
    \bмо\b                              |
    \bр[-–]?н\.?                        |
    (?<=\s)район$                       |
    (?<=\s)округ$                       |

    # субъекты федерации
    \bобласть\b                         |
    \bкрай\b                            |
    \bреспублика\b                      |
    \bфедеральный\s+округ\b             |
    \bфедеральные\s+округа\b            |
    \bавтономный\s+округ\b(?:\s*-\s*)?  |
    \bавтономная\s+область\b            |

    # посёлки, станицы, ЗАТО
    \bпос[её]л[о]к(?:\s+городского\s+типа)? |
    \bрабочий\s+пос[её]л[о]к             |
    \bстаница\b                          |
    \bЗАТО\b                             |

    # город и аббревиатуры
    \bгород(?:-герой|-курорт)?\b        |  # только отдельное слово
    \bг\.?(?=\s|$)                      |  
    \bгор\.(?=\s|$)                     |

    # в том числе:
    \bв\s+том\s+числе:
''', re.I | re.X)

def extract_toponym(series: pd.Series) -> pd.Series:

    s = series.fillna('')

    leave = s.str.match(LEAVE_PATTERN)

    # базовая очистка
    cleaned = (
        s
        .str.replace(SERVICE_PATTERN, '', regex=True)  # убираем служ. слова
        .str.replace(r'\s*\([^)]*\)', '', regex=True)  # круглые скобки
        .str.replace(r'\s{2,}', ' ', regex=True)       # пробелы
        .str.strip()                                   # обрезаем по краям
    )

    # если начинается на 'имени ', весь остаток
    mask_imeni = cleaned.str.match(r'(?i)^имени\s+')

    # первое слово
    first = cleaned.str.split(n=1).str.get(0).fillna('')
    result = first.where(~mask_imeni, cleaned)
    result = result.where(~leave, s)

    return result




dorogi['municipality'] = extract_toponym(dorogi['municipality'])
zhile['municipality']  = extract_toponym(zhile['municipality'])
kan['municipality']    = extract_toponym(kan['municipality'])
vod['municipality']    = extract_toponym(vod['municipality'])

dorogi['region_name'] = extract_toponym(dorogi['region_name'])
zhile['region_name']  = extract_toponym(zhile['region_name'])
kan['region_name']    = extract_toponym(kan['region_name'])
vod['region_name']    = extract_toponym(vod['region_name'])


nas['region_name']  = extract_toponym(nas['region_name'])
dem['region_name']    = extract_toponym(dem['region_name'])
ras['region_name']    = extract_toponym(ras['region_name'])
vpotr['region_name']  = extract_toponym(vpotr['region_name'])
tpotr['region_name']    = extract_toponym(tpotr['region_name'])
epotr['region_name']    = extract_toponym(epotr['region_name'])
inf['region_name']  = extract_toponym(inf['region_name'])
rev['region_name']    = extract_toponym(rev['region_name'])


nas['municipality']  = extract_toponym(nas['municipality'])
dem['municipality']    = extract_toponym(dem['municipality'])
ras['municipality']    = extract_toponym(ras['municipality'])
vpotr['municipality']  = extract_toponym(vpotr['municipality'])
tpotr['municipality']    = extract_toponym(tpotr['municipality'])
epotr['municipality']    = extract_toponym(epotr['municipality'])
inf['municipality']  = extract_toponym(inf['municipality'])
rev['municipality']    = extract_toponym(rev['municipality'])

bud['region_name']  = extract_toponym(bud['region_name'])
rod['region_name']    = extract_toponym(rod['region_name'])
ded['region_name']    = extract_toponym(ded['region_name'])
deti['region_name']  = extract_toponym(deti['region_name'])
inv['region_name']    = extract_toponym(inv['region_name'])
srnas['region_name']    = extract_toponym(srnas['region_name'])
kvart['region_name']  = extract_toponym(kvart['region_name'])
zar['region_name']    = extract_toponym(zar['region_name'])
active['region_name']  = extract_toponym(active['region_name'])

bud['municipality']  = extract_toponym(bud['municipality'])
rod['municipality']    = extract_toponym(rod['municipality'])
ded['municipality']    = extract_toponym(ded['municipality'])
deti['municipality']  = extract_toponym(deti['municipality'])
inv['municipality']    = extract_toponym(inv['municipality'])
srnas['municipality']    = extract_toponym(srnas['municipality'])
kvart['municipality']  = extract_toponym(kvart['municipality'])
zar['municipality']    = extract_toponym(zar['municipality'])
active['municipality']  = extract_toponym(active['municipality'])

dfs1 = [

zhile[['year', 'region_name', 'municipality', 'oktmo', 'Жилье (1000 м2)']],
dorogi[['year', 'region_name',  'municipality', 'oktmo', 'Дороги (%)']],
kan[['year', 'region_name',  'municipality', 'oktmo', 'Канализация (1 м)']],
vod[['year', 'region_name',  'municipality', 'oktmo', 'Водопровод (1 метр)']],

    nas[['year', 'region_name', 'municipality','oktmo', 'Население']],
    dem[['year', 'region_name', 'municipality','oktmo', 'Прирост']],
    ras[['year', 'region_name', 'municipality','oktmo', 'Расходы']],
    vpotr[['year', 'region_name', 'municipality','oktmo', 'ПотреблениеВ']],
    tpotr[['year', 'region_name', 'municipality','oktmo', 'ПотреблениеТ']],
    epotr[['year', 'region_name', 'municipality','oktmo', 'ПотреблениеЭ']],
    inf[['year', 'region_name', 'municipality','oktmo', 'БезДорог']],
    rev[['year', 'region_name', 'municipality','oktmo', 'Прибыль']],

bud[['year', 'region_name', 'municipality','oktmo', 'Бюджет']],
rod[['year', 'region_name', 'municipality','oktmo', 'Рождаемость']],
ded[['year', 'region_name', 'municipality','oktmo', 'Смертность']],
deti[['year', 'region_name', 'municipality','oktmo', 'Родившихся']],
inv[['year', 'region_name', 'municipality','oktmo', 'Инвестиции']],
srnas[['year', 'region_name', 'municipality','oktmo', 'СреднегодН']],
kvart[['year', 'region_name', 'municipality','oktmo', 'Квартиры']],
zar[['year', 'region_name', 'municipality','oktmo', 'Зарплата']],
active[['year', 'region_name', 'municipality','oktmo', 'Земля']],
]


comb1 = reduce(lambda left, right: pd.merge(
    left, right, on=['year', 'municipality', 'oktmo', 'region_name'], how='outer'
), dfs1)


comb1.fillna(0, inplace=True)
comb1 = comb1.reset_index(drop=True)
comb1.to_csv('росстат_общий_new1.csv', index=False)
comb1

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Прибыль,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля
0,2006,Тверская,,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2006,Тверская,,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2006,Тверская,,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2006,Тверская,,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006,Тверская,,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637160,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
637161,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
637162,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
637163,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


СБОР ДАННЫХ

In [3]:
# Оценка численности населения на 1 января текущего года
df = pd.read_csv("data_Y48112027_112_v20241111.csv", sep=";")
df = df[df['mest'].isin(['Все население'])]
df = df.drop(columns=['mest', 'indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Население'})
df.to_csv("население.csv", index=False)

# Общий коэффициент естественного прироста (убыли)
df = pd.read_csv("data_Y48112009_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Прирост'})
df.to_csv("убыль.csv", index=False)

# Общий объем расходов бюджета муниципального образования
df = pd.read_csv("data_Y48313001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.groupby(["region_name", "municipality", "oktmo", "year"], as_index=False)["indicator_value"].sum()
df = df.rename(columns={'indicator_value': 'Расходы'})
df.to_csv("расходы_муницип.csv", index=False)

# Удельная величина потребления горячей воды в многоквартирных домах на одного проживающего
df = pd.read_csv("data_Y48155043_112_v20241111.csv", sep=";")
df = df[df['vidres'].isin(['Горячая вода'])]
df = df.drop(columns=['vidres','indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ПотреблениеВ'})
df.to_csv("вода_потреб.csv", index=False)

# Удельная величина потребления тепловой энергии в многоквартирных домах на 1 кв. м. общей площади
df = pd.read_csv("data_Y48155042_112_v20241111.csv", sep=";")
df = df[df['vidres'].isin(['Теплоэнергия'])]
df = df.drop(columns=['vidres', 'indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ПотреблениеТ'})
df.to_csv("тепло_потреб.csv", index=False)

# Удельная величина потребления электрической энергии в многоквартирных домах на одного проживающего
df = pd.read_csv("data_Y48155041_112_v20241111.csv", sep=";")
df = df[df['vidres'].isin(['Электроэнергия'])]
df = df.drop(columns=['vidres', 'indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ПотреблениеЭ'})
df.to_csv("электро_потреб.csv", index=False)

# Доля населения, проживающего в населенных пунктах, не имеющих регулярного автобусного
df = pd.read_csv("data_Y48155002_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'БезДорог'})
df.to_csv("нет_дорог.csv", index=False)

# Прибыль (убыток) до налогообложения прошлого года (по уточненным данным)
df = pd.read_csv("data_Y48042002_112_v20241111.csv", sep=";")
df = df[df['okfs'].isin(['Всего'])]
df = df[df['okved'].isin(['Всего'])]
df = df.drop(columns=['okfs', 'okved', 'indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Прибыль'})
df.to_csv("прибыль.csv", index=False)

# Профицит, дефицит (-) бюджета муниципального образования (местного бюджета), фактически исполнено
df = pd.read_csv("data_Y48013003_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Бюджет'})
df.to_csv("бюджет.csv", index=False)

# Число жилых квартир в расчете на 1000 человек населения — всего
df = pd.read_csv("data_Y48211002_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Квартиры'})
df.to_csv("квартиры.csv", index=False)

# Среднемесячная номинальная начисленная заработная плата работников крупных, средних 
df = pd.read_csv("data_Y48213002_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Зарплата'})
df.to_csv("зарплата.csv", index=False)

# Общая площадь земель муниципального образования
df = pd.read_csv("data_Y48006001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Земля'})
df.to_csv("земля.csv", index=False)
df

,region_name,municipality,oktmo,year,Земля
0,Алтайский край,Залесовский муниципальный округ,1512000,2022,327456.0
1,Алтайский край,Алейский муниципальный район,1601000,2006,340173.0
2,Алтайский край,Алейский муниципальный район,1601000,2007,340173.0
3,Алтайский край,Алейский муниципальный район,1601000,2008,340173.0
4,Алтайский край,Алейский муниципальный район,1601000,2009,340173.0
...,...,...,...,...,...
363955,Еврейская автономная область,Город Биробиджан,99701000,2018,16938.0
363956,Еврейская автономная область,Город Биробиджан,99701000,2019,16938.0
363957,Еврейская автономная область,Город Биробиджан,99701000,2020,16938.0
363958,Еврейская автономная область,Город Биробиджан,99701000,2021,16938.0


In [2]:
# Число спортивных сооружений
df = pd.read_csv("data_Y48003001_112_v20241111.csv", sep=";")
df = df[df['sport'].isin(['Спортивные сооружения — всего'])]
df = df.drop(columns=['sport','indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Спорт'})
df.to_csv("спорт.csv", index=False)

# Число муниципальных спортивных сооружений
df = pd.read_csv("data_Y48003002_112_v20241111.csv", sep=";")
df = df[df['sport'].isin(['Спортивные сооружения — всего'])]
df = df.drop(columns=['sport','indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'СпортМ'})
df.to_csv("спортм.csv", index=False)

# Количество предприятий по утилизации и переработке бытовых и промышленных отходов
df = pd.read_csv("data_Y48005003_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Мусор'})
df.to_csv("мусор.csv", index=False)

# Количество муниципальных предприятий по утилизации и переработке бытовых и промышленных отходов
df = pd.read_csv("data_Y48005004_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'МусорМ'})
df.to_csv("мусорм.csv", index=False)

# Общая протяженность освещенных частей улиц, проездов, набережных
df = pd.read_csv("data_Y48006003_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Свет'})
df.to_csv("свет.csv", index=False)

# Число парков культуры и отдыха (городских садов)
df = pd.read_csv("data_Y48017015_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Парки'})
df.to_csv("парки.csv", index=False)

# Число общеобразовательных организаций на начало учебного года
df = pd.read_csv("data_Y48015001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Школа'})
df.to_csv("школы.csv", index=False)

# Текущие (эксплуатационные) затраты на охрану окружающей среды, включая оплату услуг природоохранного назначения (c 2012 г. — включая индивидуальных предпринимателей)
df = pd.read_csv("data_Y48055001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'Эко'})
df.to_csv("экология.csv", index=False)
df


,region_name,municipality,oktmo,year,Эко
0,Алтайский край,Алейский муниципальный район,1601000,2008,0.0
1,Алтайский край,Алейский муниципальный район,1601000,2009,0.0
2,Алтайский край,Алейский муниципальный район,1601000,2023,1792.0
3,Алтайский край,Алтайский муниципальный район,1602000,2008,53.7
4,Алтайский край,Алтайский муниципальный район,1602000,2009,77.6
...,...,...,...,...,...
41429,Еврейская автономная область,Город Биробиджан,99701000,2019,88531.0
41430,Еврейская автономная область,Город Биробиджан,99701000,2020,90573.0
41431,Еврейская автономная область,Город Биробиджан,99701000,2021,65040.0
41432,Еврейская автономная область,Город Биробиджан,99701000,2022,44829.0


In [3]:
# Число муниципальных органов охраны общественного порядка
df = pd.read_csv("data_Y48020001_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ОхранаО'})
df.to_csv("охранао.csv", index=False)

# Число работников муниципальных органов охраны общественного порядка
df = pd.read_csv("data_Y48020002_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ОхранаР'})
df.to_csv("охранар.csv", index=False)

# Число добровольных формирований населения по охране общественного порядка
df = pd.read_csv("data_Y48020003_112_v20241111.csv", sep=";")
df = df.drop(columns=['indicator_section_code',	'indicator_section','indicator_code','indicator_name','region_id','mun_level','mun_district', 'indicator_unit','indicator_period','comment','mun_type','mun_type_oktmo','oktmo_stable','oktmo_history','oktmo_year_from','oktmo_year_to'])
df = df.rename(columns={'indicator_value': 'ДружинаО'})
df.to_csv("дружинао.csv", index=False)
df

,region_name,municipality,oktmo,year,ДружинаО
0,Алтайский край,Алейский муниципальный район,1601000,2007,4.0
1,Алтайский край,Алейский муниципальный район,1601000,2008,3.0
2,Алтайский край,Алейский муниципальный район,1601000,2009,2.0
3,Алтайский край,Алейский муниципальный район,1601000,2010,3.0
4,Алтайский край,Алейский муниципальный район,1601000,2011,3.0
...,...,...,...,...,...
120882,Еврейская автономная область,Город Биробиджан,99701000,2013,2.0
120883,Еврейская автономная область,Город Биробиджан,99701000,2014,2.0
120884,Еврейская автономная область,Город Биробиджан,99701000,2015,3.0
120885,Еврейская автономная область,Город Биробиджан,99701000,2016,3.0
